In [2]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()

ts_prices = pd.read_parquet("../data/transformed/validated_prices_2022-08-02-2024-08-02.parquet")
ts_prices


/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,price_diff,volume_sma_5,volume_change,SMA_5,SMA_20,EMA_5,EMA_20,vol_5,daily_return
0,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-02 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
1,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-03 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
2,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-04 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
3,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-05 04:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
4,5.791877e+07,165.126767,164.263333,165.19,166.503333,163.40,507545.666667,2022-08-06 00:00:00,-0.160000,5.694134e+07,0.021549,165.318000,169.103000,165.296667,165.334762,0.071554,-0.000968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,3.515373e+07,218.131900,216.960000,218.24,219.300000,215.75,604680.000000,2024-07-29 04:00:00,0.093333,4.008290e+07,-0.042438,217.978000,225.414500,218.726415,221.994869,0.292077,0.000428
728,4.068162e+07,218.405900,219.190000,218.80,220.325000,216.12,584305.000000,2024-07-30 04:00:00,0.560000,3.812887e+07,0.157249,218.240000,224.705500,218.750943,221.690596,0.329983,0.002566
729,4.842297e+07,222.344100,221.440000,222.08,223.820000,220.63,668833.000000,2024-07-31 04:00:00,3.280000,3.984794e+07,0.190291,219.064000,224.431000,219.860629,221.727682,1.710839,0.014991
730,6.112524e+07,219.477300,224.370000,218.36,224.480000,217.02,876046.000000,2024-08-01 04:00:00,-3.720000,4.441905e+07,0.262319,219.125333,223.822000,219.360419,221.406950,1.670571,-0.016751


In [11]:
# # TRAIN TEST SPLIT
# from sklearn.model_selection import train_test_split

# def train_test_split_tabular(tabular_prices, target_column_name="target_close_price_next_day", test_size=0.2, random_state=42):
#     # seperate features/targets
#     X = tabular_prices.drop(columns=[target_column_name])
#     y = tabular_prices[target_column_name]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
#     return X_train, X_test, y_train, y_test

# X_train, X_test, y_train, y_test = train_test_split_tabular(tabular_prices)
# print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
# print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
# print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
# print(f"y_test: {y_test.shape}")

In [3]:
from src.data import transform_ts_data_into_features_target


n_previous_days = 12  # make sure this constant across all uses
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)


X = features
Y = targets

Number of sequences/examples: 718
Last val: 172.82666666666665
Target val: 173.19


In [4]:
# RANDOM FOREST REGRESSOR
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

num_samples, sequence_length, num_features = X.shape
X_train_flattened = X.reshape(num_samples, sequence_length * num_features)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_flattened, Y)

y_pred = model.predict(X_train_flattened)

mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")
# Print the first few predictions and actual values
print("Predictions:", y_pred[:5])
print("Actual values:", Y[:5])


Mean Squared Error: 0.7629116531530507
Predictions: [173.20205139 172.99509949 173.702052   175.07864731 172.23541946]
Actual values: [173.19 173.03 174.55 174.15 171.52]


In [5]:
# LIGHTGBM REGRESSOR
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


num_samples, sequence_length, num_features = X.shape
X_flattened = X.reshape(num_samples, sequence_length * num_features)

# Create the LightGBM Regressor model
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

# Train the model
lgb_model.fit(X_flattened, Y)

# Predict on the test set
y_pred = lgb_model.predict(X_flattened)


# Calculate the mean squared error
mse = mean_squared_error(Y, y_pred)
print(f"\nMean Squared Error: {mse}")

# Print the first few predictions and actual values
print("Predictions:", y_pred[:5])
print("Actual values:", Y[:5])

single_example = X[0].reshape(1, -1)  # use indx to get first example with is a sequence of days [[]] and reshape/flatten it to a vector
single_prediction = lgb_model.predict(single_example)
print("\nSingle Prediction:", single_prediction)
print("Actual value:", Y[0])


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002092 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49836
[LightGBM] [Info] Number of data points in the train set: 718, number of used features: 208
[LightGBM] [Info] Start training from score 173.174972
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [21]:
# # LSTM
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


# X = features
# Y = targets

# model = Sequential()
# model.add(Bidirectional(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], X.shape[2]))))
# model.add(Dropout(0.2))
# model.add(LSTM(units=100))
# model.add(Dense(1))  # Output layer for a single prediction

# model.compile(optimizer='adam', loss='mean_squared_error')

# history = model.fit(X, Y, epochs=50, batch_size=32, validation_split=0.2)

# predictions = model.predict(X)

# # Example: print the first few predictions
# print(targets[:5])
# print(predictions[:5])


In [22]:
# # RNN
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

# model = Sequential()
# model.add(SimpleRNN(units=200, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))  # Dropout to prevent overfitting
# model.add(SimpleRNN(units=30))  # Second RNN layer
# model.add(Dense(1))  # Output layer for a single prediction

# model.compile(optimizer='adam', loss='mean_squared_error')

# history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)
# predictions = model.predict(X)

# # Example: print the first few predictions
# print(targets[:5])
# print(predictions[:5])